# Import main calculation function


In [2]:
from geopolrisk.assessment.main import (
    gprs_calc,
)  # This is a one stop function to calculate all the values of the GeoPolRisk method

## Prepare the lists


In [3]:
from geopolrisk.assessment.utils import (
    default_rmlist,
)  # This is a functions that returns the default list of metals that are available in the library database


ListofMetals = default_rmlist()
ListofAlternateMetals = [
    "Aluminium",
    "Copper",
    "Nickel",
    "Tin",
    "Zinc",
]  # If you want to use a different list of metals, you can specify them here (of course, they must be available in the library database)
# Aluminium is an exception in the database, it refers to primary metal not raw material. The raw material is bauxite


ListofCountries = [
    156,
    251,
    842,
    757,
    "EU",
]  # List of countries to be assessed, you can use the ISO 3166-1 numeric code & "EU" for European Union


ListofYear = [
    2022
]  # List of years to be assessed, you can only use the year as an integer

In [4]:
gprs_calc(
    ListofYear, ListofCountries, ListofMetals
)  # This is the main function that calculates the GeoPolRisk values for the specified list of countries, years and metals
# The function returns a pandas DataFrame with the results, which is also saved as an excel file in a folder named "ouput" in the document folder of the users home directory

Calculating the GeoPolRisk: 100%|██████████| 265/265 [01:14<00:00,  3.57it/s]


,DBID,Country [Economic Entity],Raw Material,Year,GeoPolRisk Score,GeoPolRisk Characterization Factor [eq. Kg-Cu/Kg],HHI,Import Risk,Price
0,Aluminium1562022,China,Aluminium,2022,0.003945,0.024357,0.353279,0.011166,2.527817
1,Antimony1562022,China,Antimony,2022,0.076128,0.722417,0.276644,0.275184,3.885127
2,Arsenic1562022,China,Arsenic,2022,0.000152,0.075353,0.393237,0.000387,202.750097
3,Asbestos1562022,China,Asbestos,2022,0.108588,0.081655,0.334918,0.324222,0.307868
4,Baryte1562022,China,Baryte,2022,0.001981,0.000536,0.151374,0.013085,0.110799
...,...,...,...,...,...,...,...,...,...
260,TungstenEU2022,EU,Tungsten,2022,0.071131,4.342407,0.600702,0.118412,24.994016
261,UraniumEU2022,EU,Uranium,2022,0.153928,33.051142,0.247032,0.623111,87.908247
262,VanadiumEU2022,EU,Vanadium,2022,0.181131,8.651404,0.502120,0.360732,19.554918
263,ZincEU2022,EU,Zinc,2022,0.031120,0.096968,0.138882,0.224075,1.275700


# Testing individual functions for the same list


In [5]:
import random

RandomMetal = random.choice(ListofMetals)
RandomCountry = random.choice(ListofCountries)
RandomYear = random.choice(ListofYear)

In [6]:
from geopolrisk.assessment.core import (
    HHI,
)  # This is the Herfindahl-Hirschman Index function to calculate the concentration of production in a country for a specific metal and year


ProductionQuantity, hhi = HHI(RandomMetal, RandomYear, RandomCountry)

In [7]:
from geopolrisk.assessment.core import (
    importrisk,
)  # This is the function to import risk values for a specific metal, year and country from the mapped baci data


from geopolrisk.assessment.utils import (
    mapped_baci,
)  # This is a function to get the mapped baci data, which is a pandas DataFrame with the trade data for the specified countries, years and metals

mapped_baci = (
    mapped_baci()
)  # This is a function that returns the mapped baci data, which is a pandas DataFrame with the trade data for the specified countries, years and metals


Numerator, TotalTrade, Price = importrisk(
    RandomMetal, RandomYear, RandomCountry, data=mapped_baci
)  # The functions returns the values required to calculate the import risk for the specified metal, year and country
# The values are the numerator of the import risk formula (i.e. the import share wieghted by the political stability of the exporting countries), the total trade value and the price of the metal in USD per kilogram

In [8]:
from geopolrisk.assessment.core import (
    GeoPolRisk,
)  # This is the main function to calculate the GeoPolRisk values based on the numerator, total trade, price, production quantity and Herfindahl-Hirschman Index


Values = GeoPolRisk(
    Numerator, TotalTrade, Price, ProductionQuantity, hhi
)  # This function returns a dictionary with the GeoPolRisk values for the specified metal, year and country
# The values are the GeoPolRisk Score, the characterization factor, the import risk

In [9]:
Values

(0.0529988652804223, 0.1023878702719896, 0.6330141250613962)

# Testing company level functions using same data as above


In [11]:
from geopolrisk.assessment.core import importrisk_company


# Copy the Company data.xlsx file in the geopolrisk/assessment/data folder to the database folder that is created in the document folder of the users home directory
# This function calculates the import risk values for a specific metal, year and country based on the company data in the database folder


Numerator_C, TotalTrade_C, Price_C = importrisk_company("Nickel", 2022)


Values_c = GeoPolRisk(
    Numerator_C, TotalTrade_C, Price_C, ProductionQuantity * 1000, hhi
)

  rawMaterial  partnerDesc    qty  cifvalue  period  Notes  partnerISO  \
0      Nickel      Denmark  170.0    3196.0    2022    NaN         208   
1      Nickel  Netherlands    1.0      73.0    2022    NaN         528   
2      Nickel     Portugal  893.0    2782.0    2022    NaN         620   
3      Nickel        Egypt    4.0     113.0    2022    NaN         818   
4      Nickel          USA  498.0    3355.0    2022    NaN         842   

   partnerWGI cmdCode reporterDesc  reporterISO  
0    0.326998  Nickel      Company          999  
1    0.356539  Nickel      Company          999  
2    0.327856  Nickel      Company          999  
3    0.705611  Nickel      Company          999  
4    0.507201  Nickel      Company          999  


# Testing Regional Data


In [12]:
# you can create regions or group of countries, the following is an example of how to create regions
NewRegions = {
    "EFTA": ["Iceland", "Norway", "Switzerland"],
    "NAFTA": ["Canada", "Mexico", "USA"],
}  # Liechtenstein has a customs union with Switzerland whose trade is not reported by UN Comtrade

In [13]:
from geopolrisk.assessment.utils import regions

# This function updates the regions in the library (temporary variable only) and must be instantiated for the above new regions to be used


regions(NewRegions)

In [14]:
from geopolrisk.assessment.database import databases

databases.regionslist["EFTA"]
# A quick check to see if the regions have been updated in the library

['Iceland', 'Norway', 'Switzerland']

## Main Function to run for the regional assessment


In [15]:
gprs_calc(ListofYear, ["EFTA", "NAFTA"], ListofMetals)

Calculating the GeoPolRisk: 100%|██████████| 106/106 [00:13<00:00,  8.03it/s]


,DBID,Country [Economic Entity],Raw Material,Year,GeoPolRisk Score,GeoPolRisk Characterization Factor [eq. Kg-Cu/Kg],HHI,Import Risk,Price
0,AluminiumEFTA2022,EFTA,Aluminium,2022,0.028892,0.250305,0.353279,0.081781,3.546990
1,AntimonyEFTA2022,EFTA,Antimony,2022,0.115442,0.011542,0.276644,0.417296,0.040934
2,ArsenicEFTA2022,EFTA,Arsenic,2022,0.164176,4.912686,0.393237,0.417499,12.250974
3,AsbestosEFTA2022,EFTA,Asbestos,2022,0.189662,0.236331,0.334918,0.566294,0.510155
4,BaryteEFTA2022,EFTA,Baryte,2022,0.070911,0.034209,0.151374,0.468447,0.197513
...,...,...,...,...,...,...,...,...,...
101,TungstenNAFTA2022,NAFTA,Tungsten,2022,0.228728,18.544553,0.600702,0.380767,33.193985
102,UraniumNAFTA2022,NAFTA,Uranium,2022,0.021769,5.251664,0.247032,0.088121,98.770603
103,VanadiumNAFTA2022,NAFTA,Vanadium,2022,0.214775,16.578472,0.502120,0.427737,31.602510
104,ZincNAFTA2022,NAFTA,Zinc,2022,0.010950,0.050845,0.138882,0.078843,1.901065
